In [19]:
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional, Callable, Union
from PIL import Image, ImageEnhance, ImageFilter
import matplotlib.pyplot as plt

import sys,os,os.path

import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
from sklearn.cluster import KMeans

from collections import OrderedDict

import flwr as fl
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents, ClientManager
from flwr.server.strategy import Strategy, FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset
from flwr.common import ndarrays_to_parameters, NDArrays, Scalar, Context
from flwr.common import FitRes, Parameters, parameters_to_ndarrays
from flwr.server.client_proxy import ClientProxy
from flwr.common.logger import set_logger_propagation

from enum import Enum

device = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device(device)  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {fl.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cpu
Flower 1.14.0 / PyTorch 2.5.1+cpu


In [28]:
# Wrap the above ugly code of flower client into a class

# Helper Functions

class TaskType(Enum):

    CLASSFICATION = 0
    REGRESSION = 1


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def train(net, trainloader, epochs: int, verbose=False, device = "cpu", task_type = TaskType.CLASSFICATION):
    """Train the network on the training set."""
    if task_type == TaskType.CLASSFICATION:    
        criterion = torch.nn.CrossEntropyLoss()
    elif task_type == TaskType.REGRESSION:
        criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    net.to(device)
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(images)
            if task_type == TaskType.REGRESSION:
                outputs = outputs.squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()
            total += labels.size(0)
            if task_type == TaskType.CLASSFICATION:
                correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        if verbose:
            if task_type == TaskType.CLASSFICATION:
                epoch_acc = correct / total
                print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")
            elif task_type == TaskType.REGRESSION:
                print(f"Epoch {epoch+1}: train loss {epoch_loss}")

def test(net, testloader, device = "cpu"):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    net.to(device)
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

def test_regression(net, testloader, device="cpu"):
    """Evaluate the regression model on the entire test set."""
    criterion = nn.MSELoss(reduction="sum")
    sum_of_squares, total_samples = 0.0, 0
    net.eval()
    net.to(device)
    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(device), y.to(device)
            outputs = net(x).squeeze() # NOTE: You maight want to modify this part
            sum_of_squares += criterion(outputs, y).item()
            total_samples += len(y)

    if total_samples > 0:
        avg_mse = sum_of_squares / total_samples
    else:
        avg_mse = 0.0
    avg_loss = avg_mse

    # Note that to make sure the consistence,
    # we return mse twice to match {loss, accurancy as the test function}
    return avg_loss, avg_mse ** 0.5

# Custom Client Class
class FLClient(NumPyClient):
    """A Flower client that holds its own model and training data."""

    def __init__(
        self,
        net: nn.Module,
        trainloader: DataLoader,
        valloader: DataLoader,
        device: torch.device,
        client_id: int,
        epochs: int = 1,
        task_type: TaskType = TaskType.CLASSFICATION
    ):
        super().__init__()
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.device = device
        self.client_id = client_id
        self.epochs = epochs
        self.task_type = task_type

    def get_parameters(self, config: Dict[str, Scalar]) -> List[np.ndarray]:
        """Return the current local model parameters."""
        return get_parameters(self.net)

    def fit(
        self, parameters: List[np.ndarray], config: Dict[str, Scalar]
    ) -> Tuple[List[np.ndarray], int, Dict[str, Scalar]]:
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, device=self.device, epochs=self.epochs, verbose=False, task_type = self.task_type)
        new_params = get_parameters(self.net)
        # Return partition-id in the metrics
        # The simplest way to store the model
        return new_params, len(self.trainloader.dataset), {"partition-id": self.client_id}

    def evaluate(
        self, parameters: List[np.ndarray], config: Dict[str, Scalar]
    ) -> Tuple[float, int, Dict[str, Scalar]]:
        set_parameters(self.net, parameters)
        if self.task_type == TaskType.CLASSFICATION:
            loss, accuracy = test(self.net, self.valloader, self.device)
            print(f"[Client {self.client_id}] Evaluate -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
            return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}
        elif self.task_type == TaskType.REGRESSION:
            loss, mse = test_regression(self.net, self.valloader, self.device)
            print(f"[Client {self.client_id}] Evaluate -> Loss: {mse:.4f}")
            return float(loss), len(self.valloader.dataset), {"MSE": float(mse)}

# Custom Client for House Pricing Dataset

class HousePricingClient(fl.client.NumPyClient):
    def __init__(
        self,
        net: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        test_loader: DataLoader,
        device: torch.device,
        client_id: int,
        epochs: int = 1.
    ):
        super().__init__()
        self.net = net
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        self.client_id = client_id
        self.epochs = epochs

    def get_parameters(self, config: Dict[str, Scalar]) -> List[np.ndarray]:
        return get_parameters(self.net)

    '''def set_parameters(self, parameters):
        state_dict = dict(zip(self.model.state_dict().keys(), parameters))
        self.model.load_state_dict({k: torch.tensor(v) for k, v in state_dict.items()})'''

    def fit(
        self, parameters: List[np.ndarray], config: Dict[str, Scalar]
    ) -> Tuple[List[np.ndarray], int, Dict[str, Scalar]]:
        set_parameters(self.net, parameters)
        train(self.net, self.train_loader, device=self.device, epochs=self.epochs, verbose=False)
        new_params = get_parameters(self.net)
        # Return partition-id in the metrics
        # The simplest way to store the model
        return new_params, len(self.train_loader.dataset), {"partition-id": self.client_id}
        

    def evaluate(
        self, parameters: List[np.ndarray], config: Dict[str, Scalar]
    ) -> Tuple[float, int, Dict[str, Scalar]]:
        set_parameters(self.net, parameters)
        loss, rmse = test_regression(self.net, self.val_loader, self.device)
        print(f"[Client {self.client_id}] Evaluate -> Loss: {loss:.4f}, RMSE: {rmse:.4f}")
        return float(loss), len(self.val_loader.dataset), {"RMSE": float(rmse)}


class DefaultStrategy(FedAvg):

    # A custom strategy to store all the parameters.
    # https://github.com/adap/flower/issues/487
    # https://flower.ai/docs/framework/how-to-save-and-load-model-checkpoints.html

    def __init__(self, model: type, total_round: int, only_last: bool = True, save_dir: str = "models", *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_dir = save_dir
        os.makedirs(self.save_dir, exist_ok=True)
        self.model = model
        self.total_round = total_round
        self.only_last = only_last

    def aggregate_fit(
        self,
        server_round: int,
        results: list[tuple[ClientProxy, FitRes]],
        failures: list[Union[tuple[ClientProxy, FitRes], BaseException]],
    ) -> tuple[Optional[Parameters], dict[str, Scalar]]:
        """
        Aggregate model weights using weighted average.
        Also save each client's model and the global server model.
        """

        if self.only_last and server_round < self.total_round:
            return super().aggregate_fit(server_round, results, failures)

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(
            server_round, results, failures
        )

        # For each client which returned FitRes, save the client model
        for (_, fit_res) in results:
            id_ = fit_res.metrics["partition-id"]

            client_parameters: Optional[Parameters] = fit_res.parameters
            if client_parameters is not None:
                net = self.model()
                print(f"[Round {server_round}] Saving model for client {id_}...")

                # Convert `Parameters` to `list[np.ndarray]`
                client_ndarrays : list[np.ndarray] = parameters_to_ndarrays(
                    client_parameters
                )

                # Convert `list[np.ndarray]` to PyTorch `state_dict`
                params_dict = zip(net.state_dict().keys(), client_ndarrays)
                state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
                net.load_state_dict(state_dict, strict=True)

                # Save the model to disk
                torch.save(net.state_dict(), f"{self.save_dir}/client-{server_round}-{id_}.pth")

        # If `aggregated_parameters` is not None, update the global net and save it
        if aggregated_parameters is not None:
            net = self.model()
            print(f"Saving round {server_round} aggregated_parameters...")

            # Convert `Parameters` to `list[np.ndarray]`
            aggregated_ndarrays: list[np.ndarray] = parameters_to_ndarrays(
                aggregated_parameters
            )

            # Convert `list[np.ndarray]` to PyTorch `state_dict`
            params_dict = zip(net.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            net.load_state_dict(state_dict, strict=True)

            # Save the model to disk
            torch.save(net.state_dict(), f"{self.save_dir}/server-{server_round}.pth")

        return aggregated_parameters, aggregated_metrics

In [33]:
# Use the following class to run the experiment

# You need to provide the following information:
# 1. The Network class (dont instantiate it)
#       (assume we use the same network for all clients and server)
# 2. The list of data loaders for each client,
#       where loaders is a list of loader tuples (train, val, test)
#       i.e. loaders = [ (train_loader_0, val_loader_0, test_loader_0), ... ]
#       NOTE: In fit and evaluate, we ONLY use the train_loader and val_loader,
#             But we ask you to pyt them together for simplicity for any future test use.
#       NOTE: we assume the number of clients == number of data loaders
# 3. Number of clients

# See next block for an example of how to use this class

# Set up logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
set_logger_propagation(logger, True)

class FLExperiment:
    """
    A federated learning experiment interface class.

    NOTE: For each client, we now expect a tuple of three DataLoaders:
    (train_loader, val_loader, test_loader).
    """

    def __init__(
        self,
        model_cls: type,
        client_loaders: List[Tuple[DataLoader, DataLoader, DataLoader]],
        num_clients: int,
        device: torch.device = torch.device("cpu"),
        local_epochs: int = 1,
        num_rounds: int = 5,
        task_type: TaskType = TaskType.REGRESSION,
        # strategy: Optional[Strategy] = None, # Is not supported yet. and may not be needed
    ):
        """
        Args:
            model_cls (type): A PyTorch nn.Module class (not an instance).
                We'll instantiate `model_cls()` for each client and server.
            client_loaders (List[(DataLoader, DataLoader, DataLoader)]):
                A list of (train_loader, val_loader, test_loader) for each client.
            num_clients (int): Number of clients to simulate.
            device (torch.device): CPU or GPU device.
            local_epochs (int): Local epochs on each client per round.
            num_rounds (int): How many global training rounds.
            strategy (Optional[Strategy]): Use a custom Flower strategy or fallback to default FedAvg.
        """
        logger.info("Initializing FLExperiment")
        if len(client_loaders) != num_clients:
            raise ValueError(
                f"Number of client loader tuples ({len(client_loaders)}) does not match "
                f"the number of clients ({num_clients})."
            )

        self.model_cls = model_cls
        self.client_loaders = client_loaders
        self.num_clients = num_clients
        self.local_epochs = local_epochs
        self.num_rounds = num_rounds
        self.device = device
        self.task_type = task_type

        # Store final trained models
        self._client_models: List[Optional[nn.Module]] = [None] * self.num_clients
        self._server_model: Optional[nn.Module] = None

        # Create one model per client (instantiate model_cls)
        self.client_nets = [self.model_cls().to(self.device) for _ in range(self.num_clients)]

        self.strategy = self._create_default_strategy(save_only_last=True)
        logger.info("FLExperiment initialized successfully")
        # # Use user-provided strategy or create a default one
        # if strategy is None:
        #     self.strategy = self._create_default_strategy()
        # else:
        #     self.strategy = strategy

    def _create_default_strategy(self, save_only_last: bool) -> Strategy:
        """Create a default FedAvg strategy with a minimal server_evaluate."""

        logger.debug("Creating default strategy")

        def server_evaluate(
            server_round: int,
            parameters: NDArrays,
            config: Dict[str, Scalar]
        ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
            # Minimal server eval (no real evaluation)
            net = self.model_cls().to(self.device)
            set_parameters(net, parameters)
            print(f"[Server] Round {server_round} - no global evaluation implemented.")
            return None
        
        def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
            accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
            examples = [num_examples for num_examples, _ in metrics]
            if sum(examples) == 0:
                return {"accuracy": 0.0}
            return {"accuracy": sum(accuracies) / sum(examples)}

        def weighted_average_regression(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
            total_sum_of_squares = 0.0
            total_samples = 0
            for (num_examples, m) in metrics:
                if "sum_of_squares" in m:
                    total_sum_of_squares += m["sum_of_squares"]
                    total_samples += num_examples
            if total_samples == 0:
                return {"rmse": 0.0}
            rmse = (total_sum_of_squares / total_samples) ** 0.5
            return {"rmse": rmse}
        
        if self.task_type == TaskType.CLASSFICATION:
            aggregation_fn = weighted_average
        else:
            aggregation_fn = weighted_average_regression

        default_strategy = DefaultStrategy(
            model = self.model_cls,
            total_round = self.num_rounds,
            only_last = True,
            fraction_fit=1.0,
            fraction_evaluate=1.0,
            min_fit_clients=self.num_clients,
            min_evaluate_clients=self.num_clients,
            min_available_clients=self.num_clients,
            evaluate_fn=server_evaluate,
            evaluate_metrics_aggregation_fn=aggregation_fn,
        )
        return default_strategy

    def _client_fn(self, context: Context) -> Client:
        """Construct one Flower client using the partition_id to pick (train, val, test)."""
        partition_id = context.node_config["partition-id"]
        trainloader, valloader, testloader = self.client_loaders[partition_id]
        net = self.client_nets[partition_id]
        logger.info(f"Creating client {partition_id}")

        client = HousePricingClient(
            net=net,
            train_loader=trainloader,
            val_loader=valloader,
            test_loader = testloader,
            device=self.device,
            client_id=partition_id,
            epochs=self.local_epochs
        )
        return client.to_client()

    def _server_fn(self, context: Context) -> ServerAppComponents:
        """Server-side: configure strategy and server config."""
        config = ServerConfig(num_rounds=self.num_rounds)
        return ServerAppComponents(strategy=self.strategy, config=config)

    def run(self, save_only_last: bool = True) -> None:
        """Run the federated learning simulation and store final client/server models.
        
        Args:
            save_only_last (bool): Save only the last round of models.
                Default True. If False, all models will be saved.
        """
        print("[FLExperiment] Starting federated training...")
        self.strategy = self._create_default_strategy(save_only_last=save_only_last)
        client_app = ClientApp(client_fn=self._client_fn)
        server_app = ServerApp(server_fn=self._server_fn)

        # Resource allocation
        if self.device.type == "cuda":
            backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
        else:
            backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

        # Run the simulation
        run_simulation(
            client_app=client_app,
            server_app=server_app,
            num_supernodes=self.num_clients,
            backend_config=backend_config,
        )
        print("[FLExperiment] Federated training finished.")

    def get_clients(self, round_num: int = 0) -> List[nn.Module]:
        """Return final trained models for all clients (if they have been saved).
        
        Args:
            round_num (int): Round number to fetch models from. Default 0 (last round).
        
        Returns:
            List[nn.Module]: List of final trained models for all clients.
                The index of the list corresponds to the client ID 
                and the index of dataloader. 
        """
        assert round_num <= self.num_rounds, f"Round {round_num} not available, only {self.num_rounds} rounds."
        if round_num <= 0:
            round_num = self.num_rounds
        try:
            return [
                torch.load(f"models/client-{round_num}-{cid}.pth", map_location=self.device, weights_only=True)
                for cid in range(self.num_clients)
            ]
        except FileNotFoundError:
            raise RuntimeError("Client models are not available. Have you called run() or set only_last=True?")
    
    def get_client_dataloader_tuples(self, round_num: int = 0) -> List[Tuple[nn.Module, Tuple[DataLoader, DataLoader, DataLoader]]]:
        """Return the dataloaders for all clients.
         
        Args:
            round_num (int): Round number to fetch models from. Default 0 (last round).
        
        Returns:
            List[Tuple[nn.Module, Tuple[DataLoader, DataLoader, DataLoader]]]:
                List of (client_model, (train_loader, val_loader, test_loader))
        """
        assert round_num <= self.num_rounds, f"Round {round_num} not available, only {self.num_rounds} rounds."
        if round_num <= 0:
            round_num = self.num_rounds
        try:
            clients = self.get_clients(round_num)
            return list(zip(clients, self.client_loaders)) 
        except FileNotFoundError:
            raise RuntimeError("Client dataloaders are not available. Have you called run() or set only_last=True?")

    def get_server(self, round_num: int = 0) -> nn.Module:
        """Return the final server model (if stored).

        Args:
            round_num (int): Round number to fetch models from. Default 0 (last round).
        
        Returns:
            nn.Module: The final server model.
        """
        assert round_num <= self.num_rounds, f"Round {round_num} not available, only {self.num_rounds} rounds."
        if round_num <= 0:
            round_num = self.num_rounds
        try:
            return torch.load(f"models/server-{round_num}.pth", map_location=self.device, weights_only=True)
        except FileNotFoundError:
            raise RuntimeError("Server model is not available. Have you called run() or set only_last=True?")

In [17]:
# Updated Neural Network for Tabular Data

class Net(nn.Module):
    def __init__(self, input_size = 8):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
# Data Loaders for House Pricing Dataset

def load_data(client_id):
    data_path = f"house_pricing_datasets/client_{client_id}.csv"
    df = pd.read_csv(data_path)

    # Encode categorical features
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = pd.Categorical(df[col]).codes

    # Separate features and target
    features = df.drop(columns=["House Price"]).values
    target = df["House Price"].values.reshape(-1, 1)

    # Split into train, validation, and test sets
    train_size = int(0.7 * len(features))
    val_size = int(0.1 * len(features))
    test_size = len(features) - train_size - val_size
    indices = np.random.permutation(len(features))
    train_idx, val_idx, test_idx = indices[:train_size], indices[train_size:train_size + val_size], indices[train_size + val_size:]

    X_train, y_train = features[train_idx], target[train_idx]
    X_val, y_val = features[val_idx], target[val_idx]
    X_test, y_test = features[test_idx], target[test_idx]

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Create data loaders
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=32, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

    return train_loader, val_loader, test_loader

In [ ]:
# Define loaders for all clients
NUM_CLIENTS = 5
loaders = [
    load_data(client_id) for client_id in range(1, NUM_CLIENTS + 1)
]

In [34]:
fl_exp = FLExperiment(
    model_cls=Net,
    client_loaders=loaders,
    num_clients=5,
    num_rounds=20,
    task_type = TaskType.REGRESSION
)


fl_exp.run(False)

2025-01-09 13:22:38,409 - INFO - Initializing FLExperiment
2025-01-09 13:22:38,415 - DEBUG - Creating default strategy
2025-01-09 13:22:38,419 - INFO - FLExperiment initialized successfully
2025-01-09 13:22:38,421 - DEBUG - Creating default strategy
INFO :      Starting Flower ServerApp, config: num_rounds=20, no round_timeout
INFO :      


[FLExperiment] Starting federated training...


INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


[Server] Round 0 - no global evaluation implemented.


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

(ClientAppActor pid=26560) [Client 2] Evaluate -> Loss: 12045480.0000, RMSE: 3470.6599 [repeated 20x across cluster]


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


[Round 20] Saving model for client 0...
[Round 20] Saving model for client 1...
[Round 20] Saving model for client 4...
[Round 20] Saving model for client 3...
[Round 20] Saving model for client 2...
Saving round 20 aggregated_parameters...
[Server] Round 20 - no global evaluation implemented.


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 20 round(s) in 46.56s
INFO :      	History (loss, distributed):
INFO :      		round 1: 54644372.53731343
INFO :      		round 2: 54644370.62686567
INFO :      		round 3: 54644370.14925373
INFO :      		round 4: 54644370.14925373
INFO :      		round 5: 54644372.53731343
INFO :      		round 6: 54644370.14925373
INFO :      		round 7: 54644370.14925373
INFO :      		round 8: 54644370.14925373
INFO :      		round 9: 54644370.14925373
INFO :      		round 10: 54644370.14925373
INFO :      		round 11: 54644370.14925373
INFO :      		round 12: 54644370.14925373
INFO :      		round 13: 54644370.14925373
INFO :      		round 14: 54644370.14925373
INFO :      		round 15: 54644370.14925373
INFO :      		round 16: 54644370.14925373
INFO :      		round 17: 54644370.14925373
INFO :      		round 18: 54644370.14925373
INFO :      		round 19: 54644370.14925373
INFO :      		round 2

(ClientAppActor pid=41516) [Client 1] Evaluate -> Loss: 20749964.0000, RMSE: 4555.2128 [repeated 9x across cluster]
[FLExperiment] Federated training finished.


In [36]:
# Test the client model 0 on the corresponding test set

clients = fl_exp.get_clients()
model0_1 = Net().to(DEVICE)
model0_1.load_state_dict(clients[0])
model0_1.eval()

_, _, test_loader_1 = loaders[0]
loss, rmse = test_regression(model0_1, test_loader_1)
print(f"Client 0 rmse: {rmse}")

# Or Alternatively
client_dls = fl_exp.get_client_dataloader_tuples()
model0_2 = Net().to(DEVICE)
model0_2.load_state_dict(client_dls[0][0])
model0_2.eval()

_, _, test_loader_2 = client_dls[0][1]
loss, rmse = test_regression(model0_2, test_loader_2)
print(f"Client 0 rmse: {rmse}")

Client 0 rmse: 121.39111776886217
Client 0 rmse: 14735.803473173764


c:\Users\barbi\Documents\ETH\HS 2024\Deep Learning\eth-dl-project\.venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
